In [68]:
# Decision Tree for Classification
import numpy as np

X_class = np.array(
    [
        [5, 1, 1, 0, 1],  # Hours, Attendance, Sleep, PhoneUsage, Tutoring
        [2, 0, 0, 1, 0],
        [6, 1, 1, 0.5, 0],
        [1, 0, 0, 1, 0],
        [4, 0.5, 1, 0, 1],
        [10, 0, 0, 0, 0],
    ]
)
y_class = np.array([1, 0, 1, 0, 1, 0])  # 1 = Passed, 0 = Failed


def calc_thresholds(x):
    return set((x[1:] + x[:-1]) / 2)


def class_count(data):
    classes = {x: np.count_nonzero(data == x) for x in set(data)}
    return {str(x): (y / len(data)) for x, y in classes.items()}


def calc_gini(data):
    return 1 - (sum(x**2 for x in class_count(data).values()))


def best_split(X, Y):
    features = X[:].T 
    best_feature = None
    best_threshold = None
    best_gini = 1.0  # start with worst impurity
    merged_x_y = np.hstack((X, Y.reshape(-1, 1)))  # merge the X & Y

    for feature_idx, feature in enumerate(features):
        thresholds = calc_thresholds(np.sort(feature))
        for threshold in thresholds:
            split = {
                "left": np.array(
                    [x[-1] for x in merged_x_y if x[feature_idx] < threshold]
                ),  # left split 1 - sum of Pk^2
                "right": np.array(
                    [x[-1] for x in merged_x_y if x[feature_idx] >= threshold]
                ),  # right split 1 - sum of Pk^2
            }

            ginis = np.array([calc_gini(split["left"]), calc_gini(split["right"])])

            l = np.array([len(x) for x in split.values()])

            # Sum of P(True||False) *  left & right split 1 - sum of Pk^2
            gini_split = np.sum((l / sum(l)) * ginis)

            if gini_split < best_gini:
                best_gini = gini_split
                best_threshold = threshold
                best_feature = feature_idx

    return best_threshold, best_feature

def make_leaf(y):
    values, counts = np.unique(y, return_counts=True)
    return {"type": "leaf", "prediction": values[np.argmax(counts)]}

def build_tree(X, Y, depth=0, max_depth=3, min_samples=2):
    if len(np.unique(Y)) == 1:
        return make_leaf(Y)

    if depth >= max_depth:
        return make_leaf(Y)

    if len(Y) < min_samples:
        return make_leaf(Y)
    

    threshold, feature = best_split(X, Y)

    if feature is None:  
        return make_leaf(Y)
    
    left_mask = X[:, feature] < threshold
    right_mask = ~left_mask


    left_child = build_tree(
        X[left_mask],
        Y[left_mask],
        depth=depth + 1,
        max_depth=max_depth,
        min_samples=min_samples
    )

    right_child = build_tree(
        X[right_mask],
        Y[right_mask],
        depth=depth + 1,
        max_depth=max_depth,
        min_samples=min_samples,
    )

    return {
        "type": "node",
        "feature": feature,
        "threshold": threshold,
        "left": left_child,
        "right": right_child,
    }


def predict_one(x, tree):
    if tree['type'] == 'leaf':
        return tree['prediction']

    feature = tree["feature"]
    threshold = tree['threshold']
    if x[feature] < threshold:
        return predict_one(x, tree['left'])
    else:
        return predict_one(x, tree['right'])

def predict(x, tree):
    return np.array([predict_one(i, tree) for i in x])

tree = build_tree(X_class, y_class)
print(tree)
X_new = np.array(
    [
        [1, 0, 0, 1, 0],
        [6, 1, 1, 0, 1],
        [3, 0.5, 1, 0, 0],
    ]
)
print(predict(X_new, tree))


{'type': 'node', 'feature': 1, 'threshold': 0.25, 'left': {'type': 'leaf', 'prediction': 0}, 'right': {'type': 'leaf', 'prediction': 1}}
[0 1 1]


In [101]:
# Decision Tree for Regression
import numpy as np

X = np.array(
    [[50, 1, 30], [70, 2, 20], [100, 3, 10], [120, 3, 5], [140, 4, 2], [160, 5, 1]]
)
Y = np.array([150, 200, 300, 360, 400, 450])

variance = np.var(Y)


def calc_threshold(x):
    uni = np.unique(x)
    return (uni[1:] + uni[:-1]) / 2


def regression_best_split(X, Y):
    best_threshold = None
    best_feature = None
    best_variance = float("inf")  # Start with infinity
    num_of_samples, num_of_features = X.shape

    for feature_idx in np.arange(num_of_features):
        feature = X[:, feature_idx]
        thresholds = calc_threshold(feature)
        for threshold in thresholds:
            left_side = Y[feature < threshold]
            right_side = Y[feature >= threshold]

            if len(left_side) == 0 or len(right_side) == 0:
                continue

            left_variance = (len(left_side) / num_of_samples) * np.var(left_side)
            right_variance = (len(right_side) / num_of_samples) * np.var(right_side)
            var_split = left_variance + right_variance

            if var_split < best_variance:
                best_feature = feature_idx
                best_variance = var_split
                best_threshold = threshold

    return best_feature, best_threshold


def make_regression_leaf(Y):
    return {"type": "leaf", "prediction": np.mean(Y)}


def regression_build_tree(X, Y, depth=0, max_depth=3, min_samples=2):
    if len(np.unique(Y)) == 1 or depth >= max_depth or len(Y) < min_samples:
        return make_regression_leaf(Y)

    feature, threshold = regression_best_split(X, Y)

    if feature is None:
        return make_regression_leaf(Y)

    left_mask = X[:, feature] < threshold
    right_mask = ~left_mask

    left_child = regression_build_tree(X[left_mask], Y[left_mask], depth=depth + 1)
    right_child = regression_build_tree(X[right_mask], Y[right_mask], depth=depth + 1)

    return {
        "type": "node",
        "feature": feature,
        "threshold": threshold,
        "left": left_child,
        "right": right_child,
    }


def reg_predict_one(x, tree):
    if tree['type'] == "leaf":
        return tree["prediction"]

    feature = tree["feature"]
    threshold = tree["threshold"]

    if x[feature] < threshold:
        return reg_predict_one(x, tree["left"])
    else:
        return reg_predict_one(x, tree["right"])

def reg_predict(x, tree):
    return [reg_predict_one(i, tree) for i in x]


reg_tree = regression_build_tree(X,Y)
new_sample = np.array([80, 2, 15])
prediction = reg_predict_one(new_sample, reg_tree)
print(prediction)


200.0


In [ ]:
# compare the custom regression tree to SKlearn regresion tree

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

np.random.seed(42)
X_large = np.sort(5 * np.random.rand(100, 1), axis=0)
Y_large = 3 * X_large.flatten() + np.random.normal(0, 1, 100)

X_train, X_test, y_train, y_test = train_test_split(
    X_large, Y_large, test_size=0.2, random_state=42
)

sk_tree = DecisionTreeRegressor(
    max_depth=3, min_samples_split=2, criterion="squared_error"
)

# My custom regression tree
reg_tree = regression_build_tree(X_train, y_train)

# sklearn fit
sk_tree.fit(X_train, y_train)

# my custom regression prediction
custom_preds = reg_predict(X_test, reg_tree)

# sklearn prediction
sk_preds = sk_tree.predict(X_test)

# MSE for my custom regression
mse_custom = mean_squared_error(y_test, custom_preds)

# MSE for SKlearn
mse_sk = mean_squared_error(y_test, sk_preds)

# R2 for my custom regression
r2_custom = r2_score(y_test, custom_preds)

# R2 for sklearn
r2_sk = r2_score(y_test, sk_preds)

print(f"Custom Tree - MSE: {mse_custom}, R2: {r2_custom}")
print(f"Sklearn Tree - MSE: {mse_sk}, R2: {r2_sk}")


Custom Tree - MSE: 0.8134723319046022, R2: 0.9597017750640147
Sklearn Tree - MSE: 0.8134723319046019, R2: 0.9597017750640147
